In [6]:
import subprocess
import os
import gudhi
import numpy as np

from csaps import csaps
from subprocess import Popen, PIPE
from numpy import genfromtxt
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
def psd_smoother(word, wordWavPath, psdPath, matlabScriptsPath): #creates the smooth psd using matlab functions.
    matlabLocation = "/Applications/MATLAB_R2018b.app/bin" #the path for matlab 
    
    myBashCommands = ["cd " + matlabLocation + ";"] #go inside matlab package
    myBashCommands[0] += "./matlab -nodesktop -r " #run with no desktop
    myBashCommands[0] += "'cd(\"" + matlabScriptsPath + "\"); "  #cd to our directory containing our matlab scripts
    myBashCommands[0] += "word_wav = audioread(\""+ wordWavPath + "\"); " #read in our word
    myBashCommands[0] += "[f, t,  psd] = GetSpectrogram(word_wav, 16000, 10); " #compute spectrogram
    myBashCommands[0] += "smooth_psd = smoothn(10*log(psd), .5); " #smooth it 
    myBashCommands[0] += "smooth_psd = SurfaceCubicInterpolator(smooth_psd, 100); " #smooth it again
    myBashCommands[0] += "dlmwrite(\"" + psdPath + word + "_smooth_psd.csv\", smooth_psd, char(\"precision\"), 10);" #save psd data
    myBashCommands[0] += "quit()'" #close matlab

    subprocess.run(myBashCommands,stderr=PIPE, stdout=PIPE, shell = True, check = True) #run all of our above bash commands

In [7]:
myPath = '/Users/luketrujillo/Desktop/Senior_year/Spring/Seminar_in_Diff_Geo/143_midterm_project/3d_spectrogram_surfaces/' #Path containing this notebook; change this

def word_to_surface(word, myPath, accent='us'): #given a word, download the .mp3 from oxford and return the signal and frequency.
    myPath = '/Users/luketrujillo/Desktop/Senior_year/Spring/Seminar_in_Diff_Geo/143_midterm_project/3d_spectrogram_surfaces/'
    
    ## Setting paths for later; keeps things organized.
    wavPath = myPath + "wavs/"                      #full path for the the .wav files
    psdPath = myPath + "psd_data/"                  #full path for the psd data 
    matlabScriptsPath = myPath + "matlab_scripts/"  #full path to get to the matlab scripts
    
    ## Now we get the words from the web and compute the psd
    url = 'https://ssl.gstatic.com/dictionary/static/sounds/oxford/{}--_{}_1.mp3'.format(word, accent) #warning: the oxford website could change their website formats for .mp3s in the future; may be a bug source in the future
    wavFileName = wavPath + '{}-{}'.format(word, accent)
    
    print(wavFileName)
    if not os.path.isfile(wavFileName + '.wav'):            #if we haven't made the .wav before, then make it
        wget.download(url, wavFileName + '.gmp3')            #download the .mp3
        sound = AudioSegment.from_mp3(wavFileName + '.mp3') 
        sound.export(wavFileName + '.wav', format="wav")    #the .wav is now created. 
    
    psdCSV = psdPath + word + "_smooth_psd.csv" #our psd data
    if not os.path.isfile(psdCSV):              #if we haven't made the .csv before, then make it
        wordWavPath = wavFileName + ".wav"
        psd_smoother(word, wordWavPath, psdPath, matlabScriptsPath,) #call helper function to compute psd
    
    word_psd = genfromtxt(psdCSV, delimiter=',')
    
    print(word_psd.shape)
    ## finally, we plot the surface
    xdata = [np.linspace(0, 81, 81), np.linspace(0, 100, 100)] #our x-y coordinates
    i,j = np.meshgrid(*xdata, indexing = "ij")              #creates the 81x100 grid
#     zdata_s = word_psd
    zdata_s = csaps(xdata, word_psd, xdata, smooth=0.3)   #using csaps to smooth our surface
    
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_facecolor('none')
    c = [s['color'] for s in plt.rcParams['axes.prop_cycle']]
    ax.scatter(j, i, zdata_s, linewidths=0.5, color=c[0], alpha=0.5)
    ax.plot_surface(j, i, zdata_s, color=c[1], linewidth=0, alpha=1)
    ax.view_init(elev=25, azim=40)
    
    plt.title("\'" + word + "\'" + " PSD surface")
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Time ')
    ax.set_zlabel('PSD')

    plt.show()
    
    return zdata_s

In [8]:
def surface_coordinates(z_data, lowerbd): #nonzero surface data
    X = [] #initialize our coordinates
    Y = []
    Z = []
    for j in range(0, 81):
        for k in range(0, 100):
            if z_data[j][k] > lowerbd:
                X.append(j)
                Y.append(k)
                Z.append(z_data[j][k])
    return X, Y, Z

def surface_pers_diag(X,Y,Z, plotPers = False):
    points = []
    for i in range(0, len(X)):
        points.append([ X[i], Y[i], Z[i] ])                                    #Creates a list of triples of 3D points 
    rips = gudhi.RipsComplex(points, max_edge_length=42)                       #Creates ripser complex
    simplex_tree = rips.create_simplex_tree(max_dimension=2)                   #creates simplices
    diag = simplex_tree.persistence(homology_coeff_field=2, min_persistence=0) #computes persistence diagram
    
    if plotPers:
        gudhi.plot_persistence_diagram(diag) #plots the points
        plt.show()                          #graphs the points
    return diag


#We put together our previous two functions to create a general one which plots the surface and the persistence diagram.

def plot_and_pers_diagram(matrix, lowerbd, show3Dplot = False, plotPers = False): #the lowerbd is the minimum grayscale value which we'll care about
    if show3Dplot:                  #Set plotme = True if you want to see the 3D representation of the surface.
        Xs = np.linspace(0, 81, 81) #initialize our coordinates
        Ys = np.linspace(0, 100, 100)
        xdata = [Xs, Ys] #our x-y coordinates
        i,j = np.meshgrid(*xdata, indexing = "ij")  #creates the 28x28 grid

        fig = plt.figure(figsize=(15, 10)) #creates a large plotting area
        ax = plt.axes(projection='3d')     #creates axes                               

        cm = plt.get_cmap("rainbow")       #color theme is "rainbow"; can change e.g. "spring", "gist_gray", "hsv"                          
        col = []                           #we want color to vary by height. So we create different colors for each point.
        for m in range(81):                #iterating through all of our points,
            for n in range(100):
                col.append( cm(float(matrix[m][n])/255)) #we set the color for each point by normalizing by max height 255.
        ax.scatter(i, j, matrix, c=col) #plots the surface
        ax.view_init(elev=25, azim=40)
    
    X,Y,Z = surface_coordinates(matrix, lowerbd)        #creates the coordinates of the surface
    diag = surface_pers_diag(X,Y,Z, plotPers)           #plots the persistence diagram of the surface
    return diag

In [1]:
zdata_s = word_to_surface("chicken", myPath) #potato
# plot_and_pers_diagram(zdata_s, -160, True, True) #monstrosity

NameError: name 'word_to_surface' is not defined

### Generalize to .wav Files

In [ ]:
myPath = '/Users/luketrujillo/Desktop/Senior_year/Spring/Seminar_in_Diff_Geo/143_midterm_project/3d_spectrogram_surfaces/' #Path containing this notebook; change this

def word_to_surface(wavFileName, myPath, accent='us'): #given a word, download the .mp3 from oxford and return the signal and frequency.
    myPath = '/Users/luketrujillo/Desktop/Senior_year/Spring/Seminar_in_Diff_Geo/143_midterm_project/3d_spectrogram_surfaces/'
    
    ## Setting paths for later; keeps things organized.
    wavPath = myPath + "wavs/"                      #full path for the the .wav files
    psdPath = myPath + "psd_data/"                  #full path for the psd data 
    matlabScriptsPath = myPath + "matlab_scripts/"  #full path to get to the matlab scripts

    print(wavFileName)
    
    if not os.path.isfile(wavFileName + '.wav'):            #if we haven't made the .wav before, then make it
        wget.download(url, wavFileName + '.gmp3')            #download the .mp3
        sound = AudioSegment.from_mp3(wavFileName + '.mp3') 
        sound.export(wavFileName + '.wav', format="wav")    #the .wav is now created. 
    
    psdCSV = psdPath + word + "_smooth_psd.csv" #our psd data
    if not os.path.isfile(psdCSV):              #if we haven't made the .csv before, then make it
        wordWavPath = wavFileName + ".wav"
        psd_smoother(word, wordWavPath, psdPath, matlabScriptsPath,) #call helper function to compute psd
    
    word_psd = genfromtxt(psdCSV, delimiter=',')
    
    print(word_psd.shape)
    ## finally, we plot the surface
    xdata = [np.linspace(0, 81, 81), np.linspace(0, 100, 100)] #our x-y coordinates
    i,j = np.meshgrid(*xdata, indexing = "ij")              #creates the 81x100 grid
#     zdata_s = word_psd
    zdata_s = csaps(xdata, word_psd, xdata, smooth=0.3)   #using csaps to smooth our surface
    
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_facecolor('none')
    c = [s['color'] for s in plt.rcParams['axes.prop_cycle']]
    ax.scatter(j, i, zdata_s, linewidths=0.5, color=c[0], alpha=0.5)
    ax.plot_surface(j, i, zdata_s, color=c[1], linewidth=0, alpha=1)
    ax.view_init(elev=25, azim=40)
    
    plt.title("\'" + word + "\'" + " PSD surface")
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Time ')
    ax.set_zlabel('PSD')

    plt.show()
    
    return zdata_s